In [ ]:
import os
import pandas as pd
from PIL import Image as pimg
import numpy as np
pimg.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# change paths and copy data if on google colab
is_colab = False
cover_path = 'CoverArts/'
df_path = 'output.csv'
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cp '/content/gdrive/MyDrive/CoverArts/' '/content/CoverArts/' -R
    cover_path = '/content/CoverArts/'
    df_path = '/content/gdrive/MyDrive/output.csv'
    is_colab = True
except:
    print('Not on Google Colab!')

if not os.path.isdir(cover_path):
    raise Exception(f'Expected directory {cover_path}')

if not os.path.isfile(df_path):
    raise Exception(f'Expected file {df_path}')

In [ ]:
import pandas as pd

df = pd.read_csv(df_path);
df.head()

In [ ]:
file_names = os.listdir('/content/CoverArts/')
print(f'Found {len(file_names)} files in /content/CoverArts/')

dictionary = dict()
fails = 0
for i, filename in enumerate(file_names):
    try:
        if not filename[-4:] == '.png':
            raise Exception('Invalid file extension')
        number = int(filename[:-4])
        data = pimg.open(f'/content/CoverArts/{filename}').convert('RGB')
        width, height = data.size
        if not width == 150 or not height == 150:
            raise Exception('Invalid size of image')
        data = np.array(data)
        dictionary[number] = data
    except Exception as e:
        print(f'Failed to load image {filename}, i: {i}, exception: {e}')
        fails = fails + 1
print(f'Failed to load {fails} images')

In [ ]:
set_file_names = set(file_names)
count = len(df.index)
df.drop(df[~df['CoverArtID'].isin(dictionary)].index, inplace=True)
count = count - len(df.index)
df.sort_values(by='CoverArtID', inplace=True)
print(f'{count} cover arts missing!')
if not len(df.index) == len(dictionary):
    raise Exception('Count of images loaded is different from expected value')

df.head()

In [ ]:
Y_data = df.drop(columns=[ 'GroupID', 'ReleaseGUID','CoverArtID', 'ImageType', 'GenreCount', 'Class']).to_numpy()
print(Y_data.shape)
print(Y_data[:10])

In [ ]:
elements = list(dictionary.items())

elements.sort(key=lambda x: x[0])
print(list(element[0] for element in elements[:50]))
X_data = np.asarray(list(element[1] for element in elements))
print(X_data.shape)

In [ ]:
savepath = 'training_data.npz'
if is_colab:
    savepath = '/content/gdrive/MyDrive/' + savepath

np.savez_compressed(savepath, X=X_data, Y=Y_data)